# Wikipedia trail

- Look at the page
[Wikipedia:Getting_to_Philosophy](https://en.wikipedia.org/wiki/Wikipedia:Getting_to_Philosophy)
on the right you can see a script doing the crawl.
- Do a Google search for the script ```scraper.py```

You should get to [this page](https://medium.com/tech-insider/task-crawling-first-link-of-wikipedia-recursively-to-reach-the-philosophy-page-9f5bcc770a60).
Now copy the script and try to get it to work!!!

---

## Discussion

Web pages are created from databases by robots.
Sometimes the format of the web page changes 
and so the crawler won't find what you are looking for.
This script no longer works but it should be easy
for you to fix it.

---

What-back-end-infrastructure for [wikipedia](https://www.quora.com/What-back-end-infrastructure-technologies-does-Wikipedia-use)

Probably Wikimedia servers is a good place to start looking at how the backend works. Its pretty complex. Some of the components include

- Master SQL database running MariaDB.
- Several Slave databases also running MariaDB.
- Application Servers running Apache and MediaWiki.
- HTTP Cache servers which serve pre compiled pages. Running Varnish HTTP Cache
- Load balancing systems. running LVS





In [6]:
import time, re
import urllib

#import bs4
import requests

start_url = "https://en.wikipedia.org/wiki/Special:Random"
target_url = "https://en.wikipedia.org/wiki/Philosophy"

# I changed so I don't use bs4
# extract paragraphs
pp = re.compile('<p>(.*?)</p>', re.DOTALL)
pax = re.compile(r'<a href="(.*?)".*?>(.*?)</a>', re.DOTALL)
# this is a better pattern extract links outside of the page
pax = re.compile(r'<a href="([^#]*?)".*?>(.*?)</a>', re.DOTALL)

def find_first(url):
    # this function defines 2 functions
    # in its **local** scope 
    
    def bad_lk(x):
        return ':' in x
        return ('Help:' in x) or ('File:' in x)
    
    def remove_text_between_parens(text):
        #https://stackoverflow.com/questions/37528373/
        n = 1  
        while n:
            # remove non-nested/flat balanced parts
            text, n = re.subn(r'\([^()]*\)', '', text) 
        return text
    
    response = requests.get(url)

    html = remove_text_between_parens(response.text)
    
    # sometimes there are none like for Categories
    paras = pp.findall(html)
    if not paras: return
    
    # this is a primitive filter 
    # I added this  so that we never get to Categories
    lks = [lk for lk in pax.findall(''.join(paras)) if not bad_lk(lk[0])]
    if not lks: return
    
    article_link = lks[0]
    print(article_link)
    
    #I wouldn't import urllib just to do this
    return urllib.parse.urljoin('https://en.wikipedia.org/', article_link[0])

def continue_crawl(search_history, target_url, max_steps=25):
    if search_history[-1] == target_url:
        print("We've found the target article!")
        return False
    elif len(search_history) > max_steps:
        print("The search has gone on suspiciously long, aborting search!")
        return False
    elif search_history[-1] in search_history[:-1]:
        print("We've arrived at an article we've already seen, aborting search!")
        return False
    else:
        return True

article_chain = [start_url]

while continue_crawl(article_chain, target_url): 
    
    first_link = find_first(article_chain[-1])
    if not first_link:
        print("We've arrived at an article with no links, aborting search!")
        break

    article_chain.append(first_link)
    # slow down otherwise wiki server will block you
    time.sleep(2) 

('/wiki/Moth', 'moth')
('/wiki/Paraphyly', 'paraphyletic')
('/wiki/Taxonomy_', 'taxonomy')
('/wiki/Categorization', 'categorization')
('/wiki/Experience', 'experience')
('/wiki/Consciousness', 'conscious')
('/wiki/Sentience', 'sentience')
('/wiki/Feeling', 'feelings')
('/wiki/Subjectivity', 'subjective')
('/wiki/Philosophy', 'philosophical')
We've found the target article!


In [ ]:
url = "https://en.wikipedia.org/wiki/Epistemology"

pax = re.compile(r'<a href="([^#]*?)".*?>(.*?)</a>', re.DOTALL)

response = requests.get(url)
html = response.text

html = re.sub('\(.*\)', '', response.text)


pax.findall(' '.join([ x for x in pp.findall(html) ]) )
#if 'wiki' in x[0]]


In [15]:
txt = ''.join(pp.findall(html))
pax.findall(txt)

[('/wiki/Religious_philosophy', 'Traditions by religion'),
 ('/wiki/Outline_of_philosophy', 'branch of philosophy'),
 ('/wiki/Knowledge', 'knowledge'),
 ('/wiki/Ethics', 'ethics'),
 ('/wiki/Logic', 'logic'),
 ('/wiki/Metaphysics', 'metaphysics'),
 ('/wiki/Justification_', 'epistemic justification'),
 ('/wiki/Reason', 'rationality'),
 ('/wiki/Belief', 'belief'),
 ('/wiki/Episteme', '<i>epistēmē</i>'),
 ('/wiki/Discourse', 'discourse'),
 ('/wiki/Johann_Fichte', 'Johann Fichte'),
 ('/wiki/Bernard_Bolzano', 'Bernard Bolzano'),
 ('/wiki/Jean_Paul', 'Jean Paul'),
 ('/wiki/James_Frederick_Ferrier', 'James Frederick Ferrier'),
 ('/wiki/Ontology', 'ontology'),
 ('/wiki/Philosophy_of_science', 'philosophy of science'),
 ('/wiki/%C3%89mile_Meyerson', 'Émile Meyerson'),
 ('/wiki/John_Locke', 'John Locke'),
 ('/wiki/An_Essay_Concerning_Human_Understanding',
  'Essay Concerning Human Understanding'),
 ('/wiki/Ancient_Greek_philosophy', 'Ancient Greek philosophers'),
 ('/wiki/Plato', 'Plato'),
 ('/wi

In [108]:
! ../.g

[master 9cbf06d] web
 1 file changed, 84 insertions(+), 182 deletions(-)
Counting objects: 4, done.
Delta compression using up to 12 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.64 KiB | 838.00 KiB/s, done.
Total 4 (delta 3), reused 0 (delta 0)
remote: Resolving deltas: 100% (3/3), completed with 3 local objects.
To https://github.com/macbuse/macbuse.github.io.git
   3057a2c..9cbf06d  master -> master
